# TP4


In [5]:
def loadMovieLens(path='data_movielens/ml-100k'):

  # Get movie titles
  movies={}
  for line in open(path+'/u.item'):
    (id,title)=line.split('|')[0:2]
    movies[id]=title

  # Load data
  prefsUsersMovies,prefsMoviesUsers = {},{}
 
  for line in open(path+'/u.data'):
    (user,movieid,rating,ts)=line.split('\t')
    prefsUsersMovies.setdefault(user,{})
    prefsMoviesUsers.setdefault(movies[movieid],{})
    prefsUsersMovies[user][movies[movieid]]=float(rating)
    prefsMoviesUsers[movies[movieid]][user]=float(rating)
    
  return prefsUsersMovies,prefsMoviesUsers

In [6]:
import numpy as np
import random
prefsUsersMovies,prefsMoviesUsers = loadMovieLens()
data = []
for user,val in prefsUsersMovies.iteritems():
        for movie,rate in val.iteritems():
            data.append((user,movie,rate))
random.shuffle(data)
train = data[:int(len(data)*0.8)]
test_before = data[int(len(data)*0.8):]
print train[:10]
print len(test_before)
print len(train)

[('181', 'Solo (1996)', 1.0), ('243', 'Chasing Amy (1997)', 4.0), ('588', 'Sleepless in Seattle (1993)', 5.0), ('13', 'Love and Death on Long Island (1997)', 1.0), ('790', 'Perez Family, The (1995)', 3.0), ('851', 'Trainspotting (1996)', 4.0), ('699', 'Akira (1988)', 3.0), ('334', 'Batman Returns (1992)', 2.0), ('389', 'Apartment, The (1960)', 5.0), ('894', 'Kicked in the Head (1997)', 3.0)]
19939
79754


In [34]:
random.sample(xrange(1000),3)

[53, 156, 944]

In [7]:
from sklearn.metrics import mean_squared_error
train.sort(key=lambda tup: (tup[0],tup[1]))
test_before.sort(key= lambda tup: (tup[0],tup[1]))
index_users = {}
index_movies = {}
train_dict = {}
for t in train:
    if t[0] in train_dict:
        train_dict[t[0]][t[1]] = t[2]
    else :
        train_dict[t[0]] = {t[1]:t[2]}
means_by_user = {}
for t in train:
    if not t[0] in index_users:
        index_users[t[0]] = len (index_users.keys()) # on donne un ID à chaque user
        means_by_user[t[0]] = np.mean(np.array(train_dict[t[0]].values()))
    if not t[1] in index_movies:
        index_movies [t[1]] = len (index_movies.keys()) # On donne un ID à chaque film
        
    
test = []
for d in test_before:
    if (d[0] in index_users and d[1] in index_movies):
        test.append(d)


        
test_predict_avg = {}
y_train = np.array([d[2] for d in train])
y_test = np.array([d[2] for d in test])
y_avg_pred = np.array([means_by_user[d[0]] for d in test])

print "Test :",len(test)
print "Train :",len(train)
print "Mean squared error :",mean_squared_error(y_test,y_avg_pred)
a = sum((y_test - y_avg_pred)**2)/len(y_test)
b = np.average((y_test - y_avg_pred)**2)
print a,b

Test : 19903
Train : 79754
Mean squared error : 1.08413479565
1.08413479565 1.08413479565


In [74]:
from numpy.linalg import norm


def cost(index_users,index_movies,P,Q,XList,l):
    pred = np.dot(P,np.transpose(Q))
    cost = 0
    for tup in XList:
        i = index_users[tup[0]]
        j = index_movies[tup[1]]
        cost += (tup[2] - pred[i][j])**2 +l*(norm(P)**2 + norm(Q)**2)
    return cost/len(XList)
def simple_gradient_descent(index_users,index_movies,alpha,beta,nb_iter):
    N = 100
    U = len(index_users)
    M = len(index_movies)
    P = np.random.rand(U,N)
    Q = np.random.rand(M,N)
    for it in range(nb_iter):
        d = np.random.randint(len(train))
        tup = train[d]
        i = index_users[tup[0]]
        j = index_movies[tup[1]]
        dotPQ = np.dot(P[i],Q[j])
        Pprime_i = P[i] - alpha*(2*(dotPQ-tup[2])*Q[j]+2*beta*P[i])
        Qprime_j = Q[j] - alpha*(2*(dotPQ-tup[2])*P[i]+2*beta*Q[j])
        P[i] = Pprime_i
        Q[j] = Qprime_j
        if it % 50000 == 0:
            print it,cost(index_users,index_movies,P,Q,train,beta)
            print tup[2]
            print dotPQ
    return P,Q
    

In [76]:
nb_iter = 2500001
alpha = 0.0017
beta = 0.2
P,Q = simple_gradient_descent(index_users,index_movies,alpha,beta,nb_iter)

 0 17633.7099531
3.0
25.0872931466
50000 10801.2453499
3.0
3.08178282772
100000

KeyboardInterrupt: 

In [167]:

def evaluate_simple(Xtest,P,Q,index_users,index_movies):
    pred = np.dot(P,np.transpose(Q))
    cost = 0
    for tup in Xtest:
        i = index_users[tup[0]]
        j = index_movies[tup[1]]
        cost += (tup[2] - pred[i][j])**2
    return cost/len(Xtest)
print evaluate_simple(test,P,Q,index_users,index_movies)
            

1.02893275003


Meilleurs résultats que avg, mais pas de beaucoup. Ca semble pertinent d'essayer d'introduire les biais

In [78]:
from numpy.linalg import norm

def cost_biased(index_users,index_movies,P,Q,Bu,Bm,XList,l):
    pred = np.dot(P,np.transpose(Q))
    cost = 0
    for tup in XList:
        i = index_users[tup[0]]
        j = index_movies[tup[1]]
        cost += (tup[2] -(Bu[i] + Bm[j]+ pred[i][j])**2 +l*(norm(P)**2 + norm(Q)**2+norm(Bu)**2+norm(Bm)**2))
    return cost/len(XList)
def biased_gradient_descent(train,index_users,index_movies,alpha,beta,nb_iter):
    N = 100
    U = len(index_users)
    M = len(index_movies)
    P = np.random.rand(U,N)
    Q = np.random.rand(M,N)
    Busers = np.random.rand(U)
    Bmovies = np.random.rand(M)
    
    for it in range(nb_iter):
        d = np.random.randint(len(train))
        tup = train[d]
        i = index_users[tup[0]]
        j = index_movies[tup[1]]
        dotPQ = np.dot(P[i],Q[j])
        rauPQ = (dotPQ + Busers[i] + Bmovies[j]-tup[2])
        Pprime_i = P[i] - alpha*(2*rauPQ*Q[j]+2*beta*(P[i]))
        Qprime_j = Q[j] - alpha*(2*rauPQ*P[i]+2*beta*Q[j])
        Busersprime_i = Busers[i] - (alpha*2*rauPQ*Busers[i]+  2 *beta*Busers[i])
        Bmoviesprime_j = Bmovies[j] - (alpha*2*rauPQ*Bmovies[j]+  2 *beta*Bmovies[j])
        P[i] = Pprime_i
        Q[j] = Qprime_j
        Busers[i] = Busersprime_i
        Bmovies[j] = Bmoviesprime_j
        if it % 500000 == 0:
            print it,cost_biased(index_users,index_movies,P,Q,Busers,Bmovies,train,beta)
            print tup[2]
            print dotPQ + Busers[i] + Bmovies[j]
    return P,Q,Busers,Bmovies

In [69]:
alpha = 0.025
beta = 0.2
nb_iter = 25000001
Pbiased,Qbiased,Busers,Bmovies = biased_gradient_descent(train,index_users,index_movies,alpha,beta,nb_iter)

0 16657.0357356
3.0
26.2309484837
500000 3613.20447993
3.0
2.41645958748
1000000

KeyboardInterrupt: 

In [49]:
def evaluate_biased(Xtest,P,Q,Busers,Bmovies,index_users,index_movies):
    pred = np.dot(P,np.transpose(Q))
    cost = 0
    for tup in Xtest:
        i = index_users[tup[0]]
        j = index_movies[tup[1]]
        cost += (tup[2] - (pred[i][j]+Busers[i]+Bmovies[j]))**2
    return cost/len(Xtest)
print evaluate_biased(test,Pbiased,Qbiased,Busers,Bmovies,index_users,index_movies)

1.02736578829


Mêmes résultats que sans biais

A partir d'ici on recommence avec la base 1M.


In [70]:
def loadMovieLens1M(path='data_movielens/ml-1M'):

  # Get movie titles
  movies={}
  for line in open(path+'/movies.dat'):
    (id,title)=line.split('::')[0:2]
    movies[id]=title

  # Load data
  prefsUsersMovies,prefsMoviesUsers = {},{}
 
  for line in open(path+'/ratings.dat'):
    (user,movieid,rating,ts)=line.split('::')
    prefsUsersMovies.setdefault(user,{})
    prefsMoviesUsers.setdefault(movies[movieid],{})
    prefsUsersMovies[user][movies[movieid]]=float(rating)
    prefsMoviesUsers[movies[movieid]][user]=float(rating)
    
  return prefsUsersMovies,prefsMoviesUsers

In [71]:
import numpy as np
import random
prefsUsersMovies1M,prefsMoviesUsers1M = loadMovieLens1M()
data1M = []
for user,val in prefsUsersMovies1M.iteritems():
        for movie,rate in val.iteritems():
            data1M.append((user,movie,rate))
random.shuffle(data1M)
train1M = data1M[:int(len(data)*0.8)]
test_before1M = data1M[int(len(data)*0.8):]
print train1M[:10]
print len(test_before1M)
print len(train1M)

[('2320', 'Men in Black (1997)', 3.0), ('4345', 'Speechless (1994)', 2.0), ('475', 'Great Escape, The (1963)', 4.0), ('5597', 'Blob, The (1958)', 1.0), ('1426', 'Falling Down (1993)', 4.0), ('5347', "Jackie Chan's First Strike (1996)", 1.0), ('1255', 'Go Fish (1994)', 4.0), ('1268', "City Slickers II: The Legend of Curly's Gold (1994)", 1.0), ('1211', 'Third Man, The (1949)', 5.0), ('5822', 'Halloween (1978)', 3.0)]
920455
79754


In [72]:
from sklearn.metrics import mean_squared_error
train1M.sort(key=lambda tup: (tup[0],tup[1]))
test_before1M.sort(key= lambda tup: (tup[0],tup[1]))
index_users1M = {}
index_movies1M = {}
train_dict1M = {}
for t in train1M:
    if t[0] in train_dict1M:
        train_dict1M[t[0]][t[1]] = t[2]
    else :
        train_dict1M[t[0]] = {t[1]:t[2]}
means_by_user1M = {}
for t in train1M:
    if not t[0] in index_users1M:
        index_users1M[t[0]] = len (index_users1M.keys()) # on donne un ID à chaque user
        means_by_user1M[t[0]] = np.mean(np.array(train_dict1M[t[0]].values()))
    if not t[1] in index_movies1M:
        index_movies1M [t[1]] = len (index_movies1M.keys()) # On donne un ID à chaque film
        
    
test1M = []
for d in test_before1M:
    if (d[0] in index_users1M and d[1] in index_movies1M):
        test1M.append(d)


        
test_predict_avg1M = {}
y_train1M = np.array([d[2] for d in train1M])
y_test1M = np.array([d[2] for d in test1M])
y_avg_pred1M = np.array([means_by_user1M[d[0]] for d in test1M])

print "Test :",len(test1M)
print "Train :",len(train1M)
print "Mean squared error :",mean_squared_error(y_test1M,y_avg_pred1M)
a = sum((y_test1M - y_avg_pred1M)**2)/len(y_test1M)
b = np.average((y_test1M - y_avg_pred1M)**2)
print a,b

Test : 911733
Train : 79754
Mean squared error : 1.15681716132
1.15681716132 1.15681716132


In [79]:
alpha = 0.025
beta = 0.2
nb_iter = 2500001
Pbiased1M,Qbiased1M,Busers1M,Bmovies1M = biased_gradient_descent(train1M,index_users1M,index_movies1M,alpha,beta,nb_iter)

0 60820.6124772
5.0
27.2033623645
500000 17620.1808934
4.0
2.74459634687
1000000 13452.2015311
5.0
3.43441242366
1500000 11427.3387723
4.0
3.51930640928
2000000 10306.0577321
4.0
2.89215402183
2500000 9596.88907196
5.0
4.7540491735


In [80]:
print evaluate_biased(test1M,Pbiased1M,Qbiased1M,Busers1M,Bmovies1M,index_users1M,index_movies1M)

1.09349891594


# Visualisation
